In [117]:
import pandas as pd
import numpy as np


# I. DF 

## I-I. DF 불러오기

In [118]:
df_krx = pd.read_excel('./dataset_2nd/krx_price_std.xlsx', index_col=False)

c:\Users\zino0\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [119]:
df_mkcap = pd.read_excel('./dataset_2nd/mkcap_2.xlsx', index_col=0)

In [120]:
df_krx.head()

,연월,종목명,종목코드,시장명,종가,최고가(종가),최저가(종가),표준편차,첨도,공왜도,베타,Amihud,Amivest,Zeros,거래량_합계,거래량_일평균,거래대금_합계,거래대금_일평균
0,2011,DMS,068790,KOSDAQ,5510,12650,3455,4.126133,1.770364,-,-,-,-,-,107602658,433882,904389701450,3646732667
1,2011,메디포스트,078160,KOSDAQ,178000,216200,28950,5.380731,1.289890,-,-,-,-,-,109676763,442245,11665140618650,47036857333
2,2011,푸른저축은행,007330,KOSDAQ,3440,4285,3015,1.752843,35.013120,-,-,-,-,-,3358197,13541,12499863825,50402677
3,2011,KBI메탈,024840,KOSDAQ,784,1935,363,5.404948,1.706310,-,-,-,-,-,85672871,345455,59636500468,240469760
4,2011,휴맥스홀딩스,028080,KOSDAQ,5670,7920,4635,2.555815,10.595189,-,-,-,-,-,4346401,17526,30228400600,121888712


In [121]:
df_mkcap_copy = df_mkcap.copy()
df_mkcap.head()

,회사명,거래소코드,회계년도,발행주식수(연결+개별),시가총액
0,(주)CMG제약,58820,2014-12-01,86626866.0,7.900370e+10
1,(주)CMG제약,58820,2018-12-01,108887244.0,4.611375e+11
2,(주)CMG제약,58820,2019-12-01,138892244.0,5.902920e+11
3,(주)CMG제약,58820,2020-12-01,138892244.0,8.027972e+11
4,(주)CMG제약,58820,2017-12-01,108887244.0,6.794564e+11


## I-II. DF Check 및 필요 처리  

### 1. Useless 컬럼 정리 - df_krx

In [122]:
# df_krx 무쓸모 컬럼 제거
useless_krxcol = ['시장명','최고가(종가)', '최저가(종가)',
                  '첨도','공왜도', 'Amihud', 'Amivest', 'Zeros','베타',
                  '거래량_합계', '거래량_일평균','거래대금_합계', '거래대금_일평균']
df_krx.drop(columns=useless_krxcol, inplace=True)

# 컬럼 순서 변경
col_order =  ['종목명', '종목코드', '연월', '종가', '표준편차']
df_krx = df_krx.reindex(columns= col_order)



In [123]:
df_krx_copy = df_krx.copy()

### 2. columns 이름 변경 to 일치 b/w df_krx & df_mkcap

In [124]:
df_krx.rename(columns={'종목명':'회사명',
                        '종목코드':'거래소코드',
                        '연월':'연도'}, inplace=True)

In [125]:
# df_krx

### 3. dtype 일치 b/w df_krx & df_mkcap

In [126]:
# df_krx.info()

In [127]:
# df_mkcap.info()

In [90]:
# df_krx : 우선주 제거 (mkcap 기업 모두 보통주) 
    # df_mkcap.loc[df_mkcap['거래소코드'].astype('str').str.endswith('0')] # -> 확인용 (17,487 row 전부 보통주)
df_krx = df_krx.loc[df_krx['거래소코드'].str.endswith('0')]

#  krx 연도 -> 시계열
# df_krx['연도'] = pd.to_datetime(df_krx['연도'], format='%Y')


C:\Users\zino0\AppData\Local\Temp\ipykernel_20316\3558735288.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_krx['연도'] = pd.to_datetime(df_krx['연도'], format='%Y')


In [131]:
# df_mkcap 

    # '거래소코드' : int -> object(6자리)   
df_mkcap['거래소코드'] = df_mkcap["거래소코드"].apply(lambda x:str(x).zfill(6))

    # '연도' 컬럼 추가 
df_mkcap['연도'] = df_mkcap['회계년도'].dt.year
# df_mkcap['연도'] = pd.to_datetime(df_mkcap['연도'], format='%Y')


# II. 시가총액 Update 

## II-I. DF 합치기 

In [138]:
print(df_krx.info())
print('-------------------------------------')
df_mkcap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13688 entries, 0 to 13687
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   회사명     13688 non-null  object
 1   거래소코드   13688 non-null  object
 2   연도      13688 non-null  int64 
 3   종가      13688 non-null  int64 
 4   표준편차    13688 non-null  object
dtypes: int64(2), object(3)
memory usage: 534.8+ KB
None
-------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17487 entries, 0 to 17486
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   회사명           17487 non-null  object        
 1   거래소코드         17487 non-null  object        
 2   회계년도          17487 non-null  datetime64[ns]
 3   발행주식수(연결+개별)  15612 non-null  float64       
 4   시가총액          14009 non-null  float64       
 5   연도            17487 non-null  int64         
dtypes: datetime64[ns](1), float64(2)

In [141]:
# Merge -> 회사명은 merge기준에서 제외
df_update = pd.merge(df_mkcap, df_krx, on = ['거래소코드', '연도'], how='left')

## II-II. krx 종가 & mkcap 주식수 로 시가총액_y  만들기 

In [ ]:
# 코드 동일 기업이 회사명 표기법이 다름 -> df_krx의 회사명 컬럼 제거
# df_update[['회사명_x', '회사명_y']] # 확인용

df_update

In [142]:
df_update.isna().sum()

회사명_x              0
거래소코드              0
회계년도               0
발행주식수(연결+개별)    1875
시가총액            3478
연도                 0
회사명_y           4352
종가              4352
표준편차            4352
dtype: int64

In [158]:
# 4 rows 업데이트 가능....... ㅜㅜ 
df_update.loc[(~((df_update['종가'].isna()) | (df_update['발행주식수(연결+개별)'].isna())))&(df_update['시가총액'].isna())]

,회사명_x,거래소코드,회계년도,발행주식수(연결+개별),시가총액,연도,회사명_y,종가,표준편차
2873,(주)비덴트,121800,2011-06-01,4200000.0,NaN,2011,비덴트,8100.0,4.139304
5043,(주)아이티센,124500,2014-03-01,3261520.0,NaN,2014,아이티센,8200.0,5.270941
11321,(주)펄어비스,263750,2017-06-01,10266300.0,NaN,2017,펄어비스,247600.0,3.89658
15759,위더스제약(주),330350,2020-06-01,8792415.0,NaN,2020,위더스제약,21550.0,4.308314


In [160]:
# 시가총액 update
df_update['시가총액_y'] = df_update['종가'] * df_update['발행주식수(연결+개별)']
df_update['시가총액'] = df_update['시가총액'].fillna(df_update['시가총액_y'])

df_update.isna().sum()

회사명_x              0
거래소코드              0
회계년도               0
발행주식수(연결+개별)    1875
시가총액            3474
연도                 0
회사명_y           4352
종가              4352
표준편차            4352
시가총액_y          4589
dtype: int64

# III. 최종 Updated 시가총액 DataFrame

In [165]:
# Useless columns 정리
useless_update  = ['발행주식수(연결+개별)', '회사명_y','종가','표준편차', '시가총액_y']
df_update.drop(columns= useless_update, inplace=True)

# 기준 컬럼명 일치 
df_update.rename(columns={'회사명_x':'회사명'})

,회사명,거래소코드,회계년도,시가총액,연도
0,(주)CMG제약,058820,2014-12-01,7.900370e+10,2014
1,(주)CMG제약,058820,2018-12-01,4.611375e+11,2018
2,(주)CMG제약,058820,2019-12-01,5.902920e+11,2019
3,(주)CMG제약,058820,2020-12-01,8.027972e+11,2020
4,(주)CMG제약,058820,2017-12-01,6.794564e+11,2017
...,...,...,...,...,...
17482,흥구석유(주),024060,2022-12-01,8.520000e+10,2022
17483,흥구석유(주),024060,2013-12-01,3.000000e+10,2013
17484,흥구석유(주),024060,2018-12-01,4.057500e+10,2018
17485,흥구석유(주),024060,2016-12-01,5.137500e+10,2016


In [166]:
# df_update.to_excel('./mkcap_update.xlsx')